In [5]:
# la fuente del codigo es: https://github.com/sankit1/cv-tricks.com

#importa la clase conjuntoDeDatos, esta obtiene las imagenes de entrenamiento por cada subcarpeta o clases de imagenes. 
import conjuntoDeDatos
import CNN
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import os

# Agregar semilla para que la inicialización aleatoria sea consistente
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

""" tamanoLotePorIteracion (batch-size) """
tamanoLotePorIteracion = 32

#Preparar la data de entrada
#Existen dos calses, perros y gatos.
clases = ['dog','cat']
numeroClases = len(clases)

# leendo las imagenes de entrada
# datos de entrenamiento: deberia ser usado el 80% de las imagenes
# datos de validacion: deberia ser usado el 20% de las imagenes, estas imagenes deben quedar fuera del conjunto de datos de entrenamiento, este conjunto de datos es necesario para calcular la exactitud del modelo.
# datos de prueba, este conjunto de datos es utilizado para probar el modelo, despue de haber entrenado el modelo funciona bien, pero cuando las imagenes son convertidas a un tamaño muy reducido el model puede fallar, a este termino se le conoce como sobre sobreajuste(Overfitting), el sobre ajuste puede ser probocado por el fondo de las imagenes.

# tamanoDeDataDeValidacion automaticamente se usara el 20% de todas las imagenes para la validacion.
tamanoDeDataDeValidacion = 0.2

# El mamaño de las imagenes es de 128 pixeles por 128.
tamanoDeImagenes = 128

# el numero de canales representa que la imagenen tiene los tres colores RGB
numeroDeColoresPorImagen = 3

# la ruta de datos de entrenamiento es la carpeta donde se encuentran los conjuntos de imagenes de entrenamiento y validacion
rutaDeDatosDeEntrenamiento=os.path.join(os.path.realpath('.'),'imagenes/train/')
print("ruta de datos de entrenamiento "+str(rutaDeDatosDeEntrenamiento))
# Cargar las imagenes de entrenamiento y validacion con sus etiquetas en memoria usando openCV para ser usadas durante el proceso de entrenamiento.
data = conjuntoDeDatos.leerDatosDeEntrenamiento(rutaDeDatosDeEntrenamiento, tamanoDeImagenes, clases, tamanoDeDataDeValidacion)

print("Completado el proceso de lecturada de imagenes.")
print("Numero de imagenes en el conjunto de entrenamiento:\t{}".format(len(data.entrenamiento.etiquetas)))
print("Numero de imagenes en el conjunto de validacion :\t{}".format(len(data.validacion.etiquetas)))

ruta de datos de entrenamiento /notebooks/CNNcatdogs/imagenes/train/
Now going to read dog files (indiceDeClase: 0)
Now going to read cat files (indiceDeClase: 1)
Completado el proceso de lecturada de imagenes.
Numero de imagenes en el conjunto de entrenamiento:	77
Numero de imagenes en el conjunto de validacion :	19


In [6]:
session = tf.Session()
tensorDeEntrada = tf.placeholder(tf.float32, shape=[None, tamanoDeImagenes,tamanoDeImagenes,numeroDeColoresPorImagen], name='tensorDeEntrada')

## Etiquetas
tensorDeClases = tf.placeholder(tf.float32, shape=[None, numeroClases], name='tensorDeClases')
tensorDeClasesAplanado = tf.argmax(tensorDeClases, dimension=1)

In [7]:
##Parametros graficos de red
# El parametro mas importante en una red convolucional es el tamaño del filtro de cada neurona.  Por ejemplo si entrada de la neurona es una imagen de 32x32x3, esto significa que la imagen tiene una dimension de 32x32 (alto x ancho) pixeles y contiene 3 colores RGB, entonces el filtro significa que se convertiran las imagenes a 5x5x3, es decir 5x5 (alto x ancho) y 3 representa el numero de colores RGB.

#Configuracion de la primera capa oculta, el tamaño del filtro es 3 y el numero de filtros es 32

capaConvolucional1 = CNN.CNN.crearCapaConvolucional(tensorDeEntrada=tensorDeEntrada,
               numeroDeCanales=numeroDeColoresPorImagen,
               tamanoDelFiltro=3,
               numeroDeFiltros=32)

capaConvolucional2 = CNN.CNN.crearCapaConvolucional(tensorDeEntrada=capaConvolucional1,
               numeroDeCanales=numeroFiltrosCapaConvolucional1,
               tamanoDelFiltro=3,
               numeroDeFiltros=32)

capaConvolucional3= CNN.CNN.crearCapaConvolucional(tensorDeEntrada=capaConvolucional1,
               numeroDeCanales=32,
               tamanoDelFiltro=3,
               numeroDeFiltros=64)

capaPlana = CNN.CNN.crearCapaAplanada(capaConvolucional3)

capaTotalmenteConectada1 = CNN.CNN.crearCapaTotalmenteConectada(tensorDeEntrada=capaPlana,
                     num_inputs=capaPlana.get_shape()[1:4].num_elements(),
                     num_outputs=128,
                     use_relu=True)

capaTotalmenteConectada2 = CNN.CNN.crearCapaTotalmenteConectada(tensorDeEntrada=capaTotalmenteConectada1,
                     num_inputs=128,
                     num_outputs=numeroClases,
                     use_relu=False)

#Softmax: Calcula las activaciones softmax, los argumentos de la funcion son:
## logits: es un tenson no vacio.  Puede ser uno de los siguientes tipos "half, float32, float64"
## dim: la dimension softmax puede ser realidad en: el valor default es -1, el cual indica la ultima dimension.
## name: es opcional, representa el nombre de la operacion.

## retorna: un tensor, tiene el mismo tipo y forma que logits.

# la variable ResultadoDeProbabilidadPorClase guarda los valores de la ultima capa (capaTotalmenteConectada2) y obtiene por medio de la funcion softmax los valores de probabilidad en un rango de 0-1
prediccionDeProbabilidadPorClase = tf.nn.softmax(capaTotalmenteConectada2,name='prediccionDeProbabilidadPorClase')

# la variable ResultadoDeProbabilidadPorClase guarda la probabilidad de cada clase aplanda es decir en un vector.
prediccionDeProbabilidadPorClaseAplanada = tf.argmax(prediccionDeProbabilidadPorClase, dimension=1)

# tf.Session es una clase para correr operaciones de tensorflow
# Un objetivo de una sesion es encapsular el entorno en el cual los objectos (operaciones) son ejecutados y los objectos (Tensores) son evaluados.

#inicializar las variables.
session.run(tf.global_variables_initializer())

#Entropia cruzada:
entropiaCruzada = tf.nn.softmax_cross_entropy_with_logits(logits=capaTotalmenteConectada2,labels=tensorDeClases)
perdidaTotalDeErrorEnClasificacion = tf.reduce_mean(entropiaCruzada)
optimizador = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(perdidaTotalDeErrorEnClasificacion)
correccionDePrediccion = tf.equal(prediccionDeProbabilidadPorClaseAplanada, tensorDeClasesAplanado)
exactitudDelModelo = tf.reduce_mean(tf.cast(correccionDePrediccion, tf.float32))

#inicializar las variables.
session.run(tf.global_variables_initializer())

AttributeError: 'module' object has no attribute 'CNN'

In [ ]:
def mostrarProgreso(epoca, feed_dict_train, feed_dict_validate, perdidaDeValidacion):
    exactitudDeEntrenamiento = session.run(exactitudDelModelo, feed_dict=feed_dict_train)
    exactitudDeValidacion = session.run(exactitudDelModelo, feed_dict=feed_dict_validate)
    msg = "Training epoca {0} --- Exactitud del entranamiento: {1:>6.1%}, Exactitud de validacion: {2:>6.1%},  perida de validacion: {3:.3f}"
    print(msg.format(epoca + 1, exactitudDeEntrenamiento, exactitudDeValidacion, perdidaDeValidacion))

recuentoIteraciones = 0

""" batch-size (tamaño del lote por iteracion), cuando se entrena una red no se  alimenta con todo el conjunto de datos, se hace por medio de iteracion y cada iteracion tiene un tamaño, frecuentemente es 16 o 32"""

#tamanoLotePorIteracion = 32
saver = tf.train.Saver()

def entrenarModelo(totalIteraciones):
    print("---------------- FUNCION DE ENTRENAMIENTO  ------------------")
    print("numero de imagenes para entrenamiento "+str(data.entrenamiento.recuento))

    global recuentoIteraciones
    for iteracion in range(recuentoIteraciones,recuentoIteraciones + totalIteraciones):
        print("-- iteracion # "+str(iteracion+1))
        entrenamientoPaquete_imagenes, entrenamientoPaquete_etiquetas, entrenamientoPaquete_nombres, entrenamientoPaquete_clases = data.entrenamiento.siguienteLote(tamanoLotePorIteracion)
        validacionPaquete_imagenes, validacionPaquete_etiquetas, validacionPaquete_nombres, validacionPaquete_clases = data.validacion.siguienteLote(tamanoLotePorIteracion)

        feed_dict_tr = {x: entrenamientoPaquete_imagenes,tensorDeClases: entrenamientoPaquete_etiquetas}
        feed_dict_val = {x: validacionPaquete_imagenes,tensorDeClases: validacionPaquete_etiquetas}
        
        session.run(optimizador, feed_dict=feed_dict_tr)

        if iteracion % int(data.entrenamiento.recuento/tamanoLotePorIteracion) == 0:
            print("mostrar epoca "+str(int(data.entrenamiento.recuento/tamanoLotePorIteracion)))
            perdidadDeValidacion = session.run(perdidaTotalDeErrorEnClasificacion, feed_dict=feed_dict_val)

            """ epoca (epoch) una epoca contiene un conjunto de iteracion que alimentan el modelo """
            epoca = int(iteracion / int(data.entrenamiento.recuento/tamanoLotePorIteracion))

            mostrarProgreso(epoca, feed_dict_tr, feed_dict_val, perdidadDeValidacion)
            """saver.save(session, 'dogs-cats-model')"""

    recuentoIteraciones += totalIteraciones

entrenarModelo(totalIteraciones=3000)
print("end")